# Imports

In [8]:
import os
import torchcde
import torch
import tensorboard
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import loggers as pl_loggers


from node.models import EegNet

from node.runners.base import Learner


### Import P300

In [9]:
from node.dataset import P300Dataset

In [7]:
ds = P300Dataset().get_data()

In [10]:
len(ds)

2

In [11]:
X, y = ds[0], ds[1]

In [12]:
X.shape

torch.Size([56540, 40, 8])

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify = y)

In [14]:
len(X_train)

45232

In [15]:
len(X_test)

11308

In [16]:
import torch.utils.data as data

train = data.TensorDataset(X_train, y_train)
test = data.TensorDataset(X_test, y_test)

In [17]:
trainloader = data.DataLoader(train, batch_size=128, shuffle=True, num_workers=4)
testloader = data.DataLoader(test, batch_size=128, shuffle=False, num_workers=4)

counts = y_test.unique(return_counts=True)[1].to(torch.float)
class_balance = counts / counts.min()

# EegNet

In [18]:
checkpoint_callback = ModelCheckpoint(
    monitor='val_f1',
    mode = 'max',
    dirpath='logs/models/demons/EEGNET/',
    save_top_k = 3,
)

In [19]:
eegnet = EegNet(
    input_size =(X.shape[1], X.shape[2]),
    rate = 50,
    F1 = 2,
    D = 4,
)

In [20]:
learn = Learner(eegnet, lr=0.05, timestamps=False, class_weights = 1/class_balance)

/home/philip/.local/lib/python3.9/site-packages/deprecate/deprecation.py:115: FutureWarning: The `F1` was deprecated since v0.7 in favor of `torchmetrics.classification.f_beta.F1Score`. It will be removed in v0.8.
  stream(template_mgs % msg_args)


In [23]:
trainer = pl.Trainer(
    max_epochs=10,
    progress_bar_refresh_rate=1,
    gradient_clip_val=1,
    callbacks=[checkpoint_callback]
    
)
trainer.fit(learn, trainloader, val_dataloaders = testloader)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name     | Type     | Params
--------------------------------------
0 | model    | EegNet   | 584   
1 | accuracy | Accuracy | 0     
2 | f1       | F1       | 0     
--------------------------------------
584       Trainable params
0         Non-trainable params
584       Total params
0.002     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

In [22]:
results = trainer.test(learn, testloader)
base_path = "results/{}".format("person_lcde")
os.makedirs(base_path, exist_ok=True)
with open("{}/pt_ode_lstm_{}.csv".format(base_path, 128), "a") as f:
    f.write("{:03f}\n".format(results[0]["val_acc"]))

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'val_acc': 0.8012911081314087,
 'val_f1': 0.713215708732605,
 'val_loss': 0.9017326831817627}
--------------------------------------------------------------------------------
